In [42]:
import json
import requests
import time

# Disable SSL certificate verification
requests.packages.urllib3.disable_warnings()

# Fetch the JSON data from the URL without SSL verification
url = "https://127.0.0.1:2999/liveclientdata/gamestats"

# Make the request and retrieve the data
response = requests.get(url, verify=False)
data = response.json()

# Extract the gameTime value
game_time_seconds = data["gameTime"]

# Convert game_time to minutes and seconds
minutes = game_time_seconds // 60
seconds = game_time_seconds % 60

# Format minutes and seconds without decimal places
formatted_minutes = "{:.0f}".format(minutes)
formatted_seconds = "{:.0f}".format(seconds)

# Print the converted time
print("Game Time: {} minutes {} seconds".format(formatted_minutes, seconds))


Game Time: 12 minutes 43.512451171875 seconds


In [43]:
import requests

playerlistURL = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(playerlistURL, verify=False)
data = response.json()

# Calculate the sum of levels and save player names for players in "ORDER" team
order_levels = []
order_players = []
for obj in data:
    if obj.get("team") == "ORDER":
        order_levels.append(int(obj["level"]))
        order_players.append(obj["summonerName"])

# Calculate the sum of levels and save player names for players in "CHAOS" team
chaos_levels = []
chaos_players = []
for obj in data:
    if obj.get("team") == "CHAOS":
        chaos_levels.append(int(obj["level"]))
        chaos_players.append(obj["summonerName"])

# Calculate the average levels for each team
order_average = sum(order_levels) / len(order_levels)
chaos_average = sum(chaos_levels) / len(chaos_levels)

# Print the averages
print(f"Average of levels for players in the 'ORDER' team: {order_average}")
print(f"Players in the 'ORDER' team: {', '.join(order_players)}")
print(f"Average of levels for players in the 'CHAOS' team: {chaos_average}")
print(f"Players in the 'CHAOS' team: {', '.join(chaos_players)}")


Average of levels for players in the 'ORDER' team: 12.0
Players in the 'ORDER' team: Kribadubadara
Average of levels for players in the 'CHAOS' team: 10.0
Players in the 'CHAOS' team: Jax Bot


In [44]:
import requests

playerlistURL = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(playerlistURL, verify=False)
data = response.json()

# Calculate the sum of scores for players in "ORDER" team
order_scores = {}
for key in data[0]["scores"]:
    order_scores[key] = sum(obj["scores"][key] for obj in data if obj.get("team") == "ORDER")

# Calculate the sum of scores for players in "CHAOS" team
chaos_scores = {}
for key in data[0]["scores"]:
    chaos_scores[key] = sum(obj["scores"][key] for obj in data if obj.get("team") == "CHAOS")

# Print the sums
print("Sum of scores for players in 'ORDER' team:")
print(order_scores)
print("Sum of scores for players in 'CHAOS' team:")
print(chaos_scores)


Sum of scores for players in 'ORDER' team:
{'assists': 0, 'creepScore': 10, 'deaths': 0, 'kills': 1, 'wardScore': 0.3333333432674408}
Sum of scores for players in 'CHAOS' team:
{'assists': 0, 'creepScore': 0, 'deaths': 1, 'kills': 0, 'wardScore': 0.0}


In [45]:
import requests

def collect_items_by_team(json_data):
    team1_items = []
    team2_items = []

    for player in json_data:
        team = player["team"]
        items = player["items"]

        if team == "ORDER":
            team1_items.extend(item["itemID"] for item in items)
        elif team == "CHAOS":
            team2_items.extend(item["itemID"] for item in items)

    return team1_items, team2_items


# Disable SSL certificate verification
requests.packages.urllib3.disable_warnings()

# Fetch JSON data from the URL
url = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    json_data = response.json()
    team1_items, team2_items = collect_items_by_team(json_data)
    print("Team 1 Items:")
    print(team1_items)
    print("Team 2 Items:")
    print(team2_items)
else:
    print("Failed to fetch JSON data. Status code:", response.status_code)


Team 1 Items:
[4633, 3115, 3089, 4645, 3135, 4629, 3513]
Team 2 Items:
[1055, 3133, 1028, 2003, 3340]


In [46]:
import requests
import json

# Static URL for the items.json data
json_url = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/items.json"

def get_item_price(item_id):
    try:
        response = requests.get(json_url)
        data = json.loads(response.text)

        item_data = data[str(item_id)]
        total_price = item_data["shop"]["prices"]["total"]
        return total_price

    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def calculate_total_item_price(item_ids):
    total_price = 0
    for item_id in item_ids:
        price = get_item_price(item_id)
        if price is not None:
            total_price += price
    return total_price

# Calculate total item price for team 1
team1_total_price = calculate_total_item_price(team1_items)

# Calculate total item price for team 2
team2_total_price = calculate_total_item_price(team2_items)

# Print the results
print(f"Total item price for team 1: {team1_total_price} gold.")
print(f"Total item price for team 2: {team2_total_price} gold.")

Total item price for team 1: 18800 gold.
Total item price for team 2: 2000 gold.


In [30]:
type(order_players)

list

In [47]:
import requests
import json

url = "https://127.0.0.1:2999/liveclientdata/eventdata"

# Make the HTTP GET request
response = requests.get(url, verify=False)  # Set verify to False if using self-signed certificate

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Initialize counters
    team_order_dragon_kills = 0
    team_order_herald_kills = 0
    team_chaos_dragon_kills = 0
    team_chaos_herald_kills = 0

    # Iterate over the events
    for event in data["Events"]:
        event_name = event.get("EventName", "")

        if event_name == "DragonKill":
            killer_name = event.get("KillerName", "")

            if killer_name in order_players:
                team_order_dragon_kills += 1
            else:
                team_chaos_dragon_kills += 1
                

        elif event_name == "HeraldKill":
            killer_name = event.get("KillerName", "")

            if killer_name in order_players:
                team_order_herald_kills += 1
            else:
                team_chaos_herald_kills += 1

    # Print the results
    print("Team Order:")
    print("Dragon kills:", team_order_dragon_kills)
    print("Herald kills:", team_order_herald_kills)
    print()
    print("Team Chaos:")
    print("Dragon kills:", team_chaos_dragon_kills)
    print("Herald kills:", team_chaos_herald_kills)
else:
    # If the request was not successful, print the error message
    print("Error:", response.status_code, response.reason)


Team Order:
Dragon kills: 1
Herald kills: 1

Team Chaos:
Dragon kills: 0
Herald kills: 0


In [48]:
team_1 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0, "Kills": 0, "Assists": 0, "Deaths": 0, "Plates": 0, "Towers": 0, "Dragons": 0, "Heralds": 0, "Sight_wards": 0, "Control_wards": 0, "Gold_diff": 0}

In [51]:
team_1["Level"]=order_average
team_1["Minions"]=order_scores["creepScore"]
team_1["Kills"]=order_scores["kills"]
team_1["Assists"]=order_scores["assists"]
team_1["Deaths"]=order_scores["deaths"]
team_1["Gold"]=team1_total_price
team_1["Gold_diff"]=team1_total_price-team2_total_price
team_1["Dragons"]=team_order_dragon_kills
team_1["Heralds"]=team_order_herald_kills

In [52]:
team_1

{'Gold': 18800,
 'Level': 12.0,
 'Minions': 10,
 'Jungle_minions': 0,
 'Kills': 1,
 'Assists': 0,
 'Deaths': 0,
 'Plates': 0,
 'Towers': 0,
 'Dragons': 1,
 'Heralds': 1,
 'Sight_wards': 0,
 'Control_wards': 0,
 'Gold_diff': 16800}